<a href="https://colab.research.google.com/github/alexshultz/comfyui-colab/blob/main/comfyui_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Git clone the repo and install the requirements. (ignore the pip errors about protobuf)

In [1]:
#@title Connect Google Drive


OPTIONS = {}

USE_GOOGLE_DRIVE = True  #@param {type:"boolean"}
WORKSPACE = 'ComfyUI'
OPTIONS['USE_GOOGLE_DRIVE'] = USE_GOOGLE_DRIVE

if OPTIONS['USE_GOOGLE_DRIVE']:
    !echo "Mounting Google Drive..."
    %cd /

    from google.colab import drive
    drive.mount('/content/drive')

    WORKSPACE = "/content/drive/MyDrive/ComfyUI"
    %cd /content/drive/MyDrive


Mounting Google Drive...
/
Mounted at /content/drive
/content/drive/MyDrive


In [46]:
#@title Environment Setup


OPTIONS = {}

USE_GOOGLE_DRIVE = True  #@param {type:"boolean"}
UPDATE_COMFY_UI = True  #@param {type:"boolean"}
WORKSPACE = 'ComfyUI'
OPTIONS['USE_GOOGLE_DRIVE'] = USE_GOOGLE_DRIVE
OPTIONS['UPDATE_COMFY_UI'] = UPDATE_COMFY_UI

if OPTIONS['USE_GOOGLE_DRIVE']:
    !echo "Mounting Google Drive..."
    %cd /

    from google.colab import drive
    drive.mount('/content/drive')

    WORKSPACE = "/content/drive/MyDrive/ComfyUI"
    %cd /content/drive/MyDrive

![ ! -d $WORKSPACE ] && echo -= Initial setup ComfyUI =- && git clone https://github.com/comfyanonymous/ComfyUI
%cd $WORKSPACE

if OPTIONS['UPDATE_COMFY_UI']:
  !echo -= Updating ComfyUI =-
  !git pull

!echo -= Install dependencies =-
# Install xFormers for CUDA 12.4 first
!pip install xformers!=0.0.18 --index-url https://download.pytorch.org/whl/cu124
# Then install remaining requirements
!pip install -r requirements.txt

# InsightFace and onnxruntime
!echo -= Installing InsightFace and onnxruntime for IP-Adapter FaceID =-
!pip install insightface onnxruntime-gpu

# Update or install IP-Adapter custom nodes
!echo -= Updating ComfyUI_IPAdapter_plus =-
# If directory exists and is a git repo, update it; otherwise, clone fresh
!cd /content/drive/MyDrive/ComfyUI/custom_nodes && (cd ComfyUI_IPAdapter_plus && git pull || git clone https://github.com/cubiq/ComfyUI_IPAdapter_plus.git ComfyUI_IPAdapter_plus)
# Install dependencies
!if [ -f /content/drive/MyDrive/ComfyUI/custom_nodes/ComfyUI_IPAdapter_plus/requirements.txt ]; then pip install -r /content/drive/MyDrive/ComfyUI/custom_nodes/ComfyUI_IPAdapter_plus/requirements.txt; else echo "requirements.txt missing, installing manually" && pip install insightface opencv-python-headless numpy torch torchvision; fi

# Update or install ComfyUI_essentials custom nodes
!echo -= Updating ComfyUI_essentials =-
# If directory exists and is a git repo, update it; otherwise, clone fresh
!cd /content/drive/MyDrive/ComfyUI/custom_nodes && (cd ComfyUI_essentials && git pull || git clone https://github.com/cubiq/ComfyUI_essentials.git ComfyUI_essentials)
# Install dependencies
!if [ -f /content/drive/MyDrive/ComfyUI/custom_nodes/ComfyUI_essentials/requirements.txt ]; then pip install -r /content/drive/MyDrive/ComfyUI/custom_nodes/ComfyUI_essentials/requirements.txt; else echo "No additional requirements for ComfyUI_essentials"; fi


# Update or install Comfy_Dungeon custom nodes
# https://github.com/cubiq/Comfy_Dungeon
!echo -= Updating Comfy_Dungeon =-
!cd /content/drive/MyDrive/ComfyUI/custom_nodes && (cd Comfy_Dungeon && git pull || git clone https://github.com/cubiq/Comfy_Dungeon.git Comfy_Dungeon)
# No requirements.txt specified in README, assuming base ComfyUI deps suffice

# Install huggingface model dowloader
!echo -= huggingface_hub =-
!pip install huggingface_hub


!echo -= Done =-


Mounting Google Drive...
/
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive
/content/drive/MyDrive/ComfyUI
-= Updating ComfyUI =-
Already up to date.
-= Install dependencies =-
Looking in indexes: https://download.pytorch.org/whl/cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.3 MB/s eta 0:00:00
     

Download some models/checkpoints/vae or custom comfyui nodes (uncomment the commands for the ones you want)

Great tools in the AP-Adapter package
`https://github.com/cubiq/ComfyUI_IPAdapter_plus?tab=readme-ov-file`

In [38]:
import os
import requests
from tqdm import tqdm

def download_civitai_model(model_id=None, model_version_id=None, filename='', save_dir='./models', token=None, overwrite=False):
    """
    Downloads a model from CivitAI using API authentication. Supports downloading by model ID or model version ID.
    Skips download if file is unchanged based on remote size. Shows progress bar.

    Parameters:
        model_id (str): The ID of the model (optional if model_version_id is used).
        model_version_id (str): The specific version ID of the model (optional if model_id is used).
        filename (str): Desired local filename.
        save_dir (str): Directory to save the file.
        token (str): API token.
        overwrite (bool): Redownload even if the file exists.

    Returns:
        nothing
    """
    if token is None:
        raise ValueError("API token is required.")
    if not filename:
        raise ValueError("Filename is required.")
    if not model_id and not model_version_id:
        raise ValueError("Either model_id or model_version_id must be provided.")

    os.makedirs(save_dir, exist_ok=True)
    save_path = os.path.join(save_dir, filename)

    # Build the correct URL
    if model_version_id:
        url = f"https://civitai.com/api/download/models/{model_version_id}?token={token}"
    else:
        url = f"https://civitai.com/api/download/models/{model_id}?type=Model&format=SafeTensor&token={token}"

    # Check if file exists locally
    if os.path.exists(save_path) and not overwrite:
        local_size = os.path.getsize(save_path)
        response = requests.get(url, stream=True, allow_redirects=True)
        remote_size = int(response.headers.get("Content-Length", -1))

        if remote_size > 0 and local_size == remote_size:
            print(f"Skipped: {filename} is already up-to-date.")
            return
        else:
            print(f"Local size: {local_size}, Remote size: {remote_size}")
            print("Local and remote sizes differ, re-downloading...")

    # Download with progress bar
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        total = int(response.headers.get("Content-Length", 0))
        with open(save_path, "wb") as f, tqdm(
            total=total, unit='B', unit_scale=True, desc=filename, ncols=80
        ) as pbar:
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:
                    f.write(chunk)
                    pbar.update(len(chunk))
        print(f"Downloaded to: {save_path}")
    else:
        print(f"Failed to download: HTTP {response.status_code}")
        print(response.text)

token = os.getenv('CIVIT_TOKEN')  # Ensure this is correctly set in your environment
print("CIVIT_TOKEN:", token)

WORKSPACE = "/content/drive/MyDrive/ComfyUI"
%cd /content/drive/MyDrive/ComfyUI
download_civitai_model(model_id='108588', model_version_id='116887', save_dir='./models/loras', filename='15_FantasyHero_v1.safetensors', token=token)
download_civitai_model(model_id='108588', model_version_id='119757', save_dir='./models/loras', filename='15_FantasyHero_v2.safetensors', token=token)
download_civitai_model(model_id='268112', save_dir='./models/loras', filename='15_Fantasy_Interior.safetensors', token=token)
download_civitai_model(model_id='287148', save_dir='./models/loras', filename='15_Fantasy_Classes_SD.safetensors', token=token)
download_civitai_model(model_id='289865', save_dir='./models/loras', filename='15_Fantasy_Races.safetensors', token=token)

!echo -= Done =-

CIVIT_TOKEN: ef4b61091ea64845b1f6a6e9ecea114b
/content/drive/MyDrive/ComfyUI
Local size: 151111920, Remote size: 151111136
Local and remote sizes differ, re-downloading...


15_FantasyHero_v1.safetensors: 100%|██████████| 151M/151M [00:01<00:00, 148MB/s]


Downloaded to: ./models/loras/15_FantasyHero_v1.safetensors
Local size: 151117763, Remote size: 151111920
Local and remote sizes differ, re-downloading...


15_FantasyHero_v2.safetensors: 100%|█████████| 151M/151M [00:03<00:00, 47.7MB/s]


Downloaded to: ./models/loras/15_FantasyHero_v2.safetensors
Skipped: 15_Fantasy_Interior.safetensors is already up-to-date.
Skipped: 15_Fantasy_Classes_SD.safetensors is already up-to-date.
Skipped: 15_Fantasy_Races.safetensors is already up-to-date.
-= Done =-


In [44]:
#@title Direct Model Downloads

import os
import requests
from tqdm import tqdm
from google.colab import userdata


# get hugginface token
userdata.get('HF_TOKEN')

#USE_GOOGLE_DRIVE = True  #@param {type:"boolean"}
WORKSPACE = 'ComfyUI'
#OPTIONS['USE_GOOGLE_DRIVE'] = USE_GOOGLE_DRIVE

#if OPTIONS['USE_GOOGLE_DRIVE']:
if True:
    !echo "Mounting Google Drive..."
    %cd /

    from google.colab import drive
    drive.mount('/content/drive')

    WORKSPACE = "/content/drive/MyDrive/ComfyUI"
    %cd /content/drive/MyDrive/ComfyUI

def download_civitai_model(model_id=None, model_version_id=None, filename='', save_dir='./models', token=None, overwrite=False):
    """
    Downloads a model from CivitAI using API authentication. Supports downloading by model ID or model version ID.
    Skips download if file is unchanged based on remote size. Shows progress bar.

    Parameters:
        model_id (str): The ID of the model (optional if model_version_id is used).
        model_version_id (str): The specific version ID of the model (optional if model_id is used).
        filename (str): Desired local filename.
        save_dir (str): Directory to save the file.
        token (str): API token.
        overwrite (bool): Redownload even if the file exists.

    Returns:
        nothing
    """
    if token is None:
        raise ValueError("API token is required.")
    if not filename:
        raise ValueError("Filename is required.")
    if not model_id and not model_version_id:
        raise ValueError("Either model_id or model_version_id must be provided.")

    os.makedirs(save_dir, exist_ok=True)
    save_path = os.path.join(save_dir, filename)

    # Build the correct URL
    if model_version_id:
        url = f"https://civitai.com/api/download/models/{model_version_id}?token={token}"
    else:
        url = f"https://civitai.com/api/download/models/{model_id}?type=Model&format=SafeTensor&token={token}"

    # Check if file exists locally
    if os.path.exists(save_path) and not overwrite:
        local_size = os.path.getsize(save_path)
        response = requests.get(url, stream=True, allow_redirects=True)
        remote_size = int(response.headers.get("Content-Length", -1))

        if remote_size > 0 and local_size == remote_size:
            print(f"Skipped: {filename} is already up-to-date.")
            return
        else:
            print(f"Local size: {local_size}, Remote size: {remote_size}")
            print("Local and remote sizes differ, re-downloading...")

    # Download with progress bar
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        total = int(response.headers.get("Content-Length", 0))
        with open(save_path, "wb") as f, tqdm(
            total=total, unit='B', unit_scale=True, desc=filename, ncols=80
        ) as pbar:
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:
                    f.write(chunk)
                    pbar.update(len(chunk))
        print(f"Downloaded to: {save_path}")
    else:
        print(f"Failed to download: HTTP {response.status_code}")
        print(response.text)


### 1.4
## GLIGEN
# 1.4 GLIGEN Textbox: Specialized model that enables spatial control of objects in generated images by specifying bounding boxes for text prompts, optimized with FP16 precision for reduced memory usage.
!echo -= 1.4 GLIGEN Textbox =-
!wget -c https://huggingface.co/comfyanonymous/GLIGEN_pruned_safetensors/resolve/main/gligen_sd14_textbox_pruned_fp16.safetensors -O ./models/gligen/14_gligen_sd14_textbox_pruned_fp16.safetensors

## T2I-Adapter
# 1.4 T2I-Adapter Depth: Lightweight adapter (77M parameters) that provides depth-map conditioning for SD 1.4, using MiDaS depth estimation to guide image generation with spatial relationships.
#!echo -= 1.4 T2I-Adapter Depth =-
#!wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_depth_sd14v1.pth -O ./models/controlnet/14_t2iadapter_depth_sd14v1.pth
# 1.4 T2I-Adapter Segmentation: Lightweight adapter for SD 1.4 that enables semantic segmentation control, allowing region-specific content generation based on colored segment maps
#!echo -= 1.4 T2I-Adapter Segmentation =-
#!wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_seg_sd14v1.pth -O ./models/controlnet/14_t2iadapter_seg_sd14v1.pth
# 1.4 T2I-Adapter Sketch: Lightweight adapter for SD 1.4 trained with PidiNet edge detection, enabling generation from hand-drawn monochrome sketches with white outlines on black backgrounds
#!echo -= 1.4 T2I-Adapter Sketch =-
#!wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_sketch_sd14v1.pth -O ./models/controlnet/14_t2iadapter_sketch_sd14v1.pth
# 1.4 T2I-Adapter Keypose: Lightweight adapter for SD 1.4 trained with mmpose skeleton images, providing human pose control through skeleton-based conditioning
#!echo -= 1.4 T2I-Adapter Keypose =-
#!wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_keypose_sd14v1.pth -O ./models/controlnet/14_t2iadapter_keypose_sd14v1.pth
# 1.4 T2I-Adapter OpenPose: Lightweight adapter for SD 1.4 that enables pose-based control using OpenPose bone images to guide human positioning and gestures
#!echo -= 1.4 T2I-Adapter OpenPose =-
#!wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_openpose_sd14v1.pth -O ./models/controlnet/14_t2iadapter_openpose_sd14v1.pth
# 1.4 T2I-Adapter Color: Lightweight adapter for SD 1.4 trained with spatial color palettes, allowing precise control over the color scheme of generated images using 8x8 color grids
#!echo -= 1.4 T2I-Adapter Color =-
#!wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_color_sd14v1.pth -O ./models/controlnet/14_t2iadapter_color_sd14v1.pth
# 1.4 T2I-Adapter Canny: Lightweight adapter for SD 1.4 trained with canny edge detection, guiding generation to follow edge outlines for precise shape control
#!echo -= 1.4 T2I-Adapter Canny =-
#!wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_canny_sd14v1.pth -O ./models/controlnet/14_t2iadapter_canny_sd14v1.pth

## T2I Styles Model
#!wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_style_sd14v1.pth -O ./models/style_models/

## VAE
# 1.4 Waifu Diffusion v1.4 VAE kl-f8-anime2: Specialized VAE for anime-style image generation, optimized for character-focused artwork and various anime aesthetics
#!echo -= Waifu Diffusion v1.4 VAE kl-f8-anime2 =-
#!wget -c https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime2.ckpt -P ./models/vae/14_kl-f8-anime2.ckpt


### 1.5
## CLIPVision model (needed for styles model)
# 1.5 CLIP ViT-Large-Patch14: OpenAI's 428M parameter vision-language model using Vision Transformer architecture for zero-shot image classification, trained on diverse image-text pairs for robust visual understanding
!echo -= 1.5 CLIP ViT-Large-Patch14 =-
!wget -c https://huggingface.co/openai/clip-vit-large-patch14/resolve/main/pytorch_model.bin -O ./models/clip_vision/15_clip_vit14.bin

## ControlNet
# 1.5 ControlNet IP2P: Half-precision model for instruct-pix2pix image editing, allowing text-based modifications to existing images
!echo -= 1.5 ControlNet IP2P =-
!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11e_sd15_ip2p_fp16.safetensors -O ./models/controlnet/15_control_v11e_sd15_ip2p_fp16.safetensors
# 1.5 ControlNet Shuffle: Half-precision model that maintains content while allowing reorganization of image elements based on shuffled reference images
!echo -= 1.5 ControlNet Shuffle =-
!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11e_sd15_shuffle_fp16.safetensors -O ./models/controlnet/15_control_v11e_sd15_shuffle_fp16.safetensors
# 1.5 ControlNet Canny: Half-precision edge detection model that guides generation following edge outlines, ideal for maintaining specific shapes
!echo -= 1.5 ControlNet Canny =-
!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_canny_fp16.safetensors -O ./models/controlnet/15_control_v11p_sd15_canny_fp16.safetensors
# 1.5 ControlNet Depth: Half-precision depth map processor that maintains spatial relationships and perspective in generated images
!echo -= 1.5 ControlNet Depth =-
!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11f1p_sd15_depth_fp16.safetensors -O ./models/controlnet/15_control_v11f1p_sd15_depth_fp16.safetensors
# 1.5 ControlNet Inpaint: Half-precision model specialized for filling in masked areas while maintaining consistency with surrounding content
!echo -= 1.5 ControlNet Inpaint =-
!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_inpaint_fp16.safetensors -O ./models/controlnet/15_control_v11p_sd15_inpaint_fp16.safetensors
# 1.5 ControlNet Lineart: Half-precision model for colorizing and enhancing line drawings while preserving original line structures
!echo -= 1.5 ControlNet Lineart =-
!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_lineart_fp16.safetensors -O ./models/controlnet/15_control_v11p_sd15_lineart_fp16.safetensors
# 1.5 ControlNet MLSD: Half-precision model focused on detecting and preserving straight lines and geometric structures in images
!echo -= .5 ControlNet MLSD =-
!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_mlsd_fp16.safetensors -O ./models/controlnet/15_control_v11p_sd15_mlsd_fp16.safetensors
# 1.5 ControlNet NormalBAE: Half-precision model using normal maps to control lighting and surface details in generated images
!echo -= 1.5 ControlNet NormalBAE =-
!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_normalbae_fp16.safetensors -O ./models/controlnet/15_control_v11p_sd15_normalbae_fp16.safetensors
# 1.5 ControlNet OpenPose: Half-precision model that uses human pose estimation to control body positioning and gestures
!echo -= 1.5 ControlNet OpenPose =-
!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_openpose_fp16.safetensors -O ./models/controlnet/15_control_v11p_sd15_openpose_fp16.safetensors
# 1.5 ControlNet Scribble: Half-precision model that generates images from simple sketches while maintaining the basic structure
!echo -= 1.5 ControlNet Scribble =-
!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_scribble_fp16.safetensors -O ./models/controlnet/15_control_v11p_sd15_scribble_fp16.safetensors
# 1.5 ControlNet Segmentation: Half-precision model using semantic segmentation maps to control region-specific content generation
!echo -= 1.5 ControlNet Segmentation =-
!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_seg_fp16.safetensors -O ./models/controlnet/15_control_v11p_sd15_seg_fp16.safetensors
# 1.5 ControlNet SoftEdge: Half-precision model using soft edge detection for more natural-looking guided generation
!echo -= 1.5 ControlNet SoftEdge =-
!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_softedge_fp16.safetensors -O ./models/controlnet/15_control_v11p_sd15_softedge_fp16.safetensors
# 1.5 ControlNet Lineart Anime: Half-precision specialized model for anime-style line art colorization with improved results for cartoon aesthetics
#!echo -= 1.5 ControlNet Lineart Anime =-
#!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15s2_lineart_anime_fp16.safetensors -O ./models/controlnet/15_control_v11p_sd15s2_lineart_anime_fp16.safetensors
# 1.5 ControlNet Tile: Half-precision model enabling seamless texture generation and tiled outputs for creating repeating patterns
!echo -= 1.5 ControlNet Tile =-
!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11u_sd15_tile_fp16.safetensors -O ./models/controlnet/15_control_v11u_sd15_tile_fp16.safetensors

## Loras
# 1.5 Theovercomer8's Contrast Fix: LoRA for producing high-contrast, low-key images, trained on SD1.5 using offset noise technique.
!echo -= 1.5 Theovercomer8s Contrast Fix =-
!wget -c https://civitai.com/api/download/models/10638 -O ./models/loras/15_theovercomer8sContrastFix_sd15.safetensors
# FantasyHero V2 LoRA: Specialized for heroic fantasy art with dynamic compositions. Best used with ReV Animated base model.
# Trigger: "f4nt4s4y style" (replace 'a's with 4s)
# Parameters V2:
#   - Weight: 0.6 (subtle integration) to 0.8 (bold style)
#   - Sampler: DPM++ 2M Karras (faster) / DPM++ 2M SDE Karras (more detailed)
#   - Steps: 25 (sweet spot for quality/speed balance)
#   - Highres: 2x Latent upscaler (denoise 0.35-0.45)
#   - CFG: 6 (balanced creativity/accuracy)
!echo -= FantasyHero V2 LoRA =-
download_civitai_model(model_id='108588', model_version_id='119757', save_dir='./models/loras', filename='15_FantasyHero_v2.safetensors', token=token)
# FantasyRobes-10 LoRA: Adds intricate and detailed fantasy robes to characters. Enhances fabric texture, embellishments, and overall magical appearance.
# Trigger: "FanRo"
!echo -= FantasyRobes-10 LoRA =-
download_civitai_model(model_id='268112', save_dir='./models/loras', filename='15_FantasyRobes-10.safetensors', token=token)
# Fantasy Interior LoRA: Creates detailed and atmospheric fantasy interior scenes, including taverns, castles, and wizard towers.
# Trigger: "faninter"
!echo -= Fantasy Interior LoRA =-
download_civitai_model(model_id='295565', save_dir='./models/loras', filename='15_Fantasy_Interior.safetensors', token=token)
# Fantasy Classes LoRA: Specializes in generating images of characters belonging to various fantasy classes. Use with class-specific keywords for best results.
# Trigger: "Fantasy classes + [keyword]" (e.g., "Fantasy classes fighter")
# Available Keywords: fighter, wizard, mage, sorcerer, druid, ranger, rogue, samurai, alchemist, artificer
!echo -= Fantasy Classes LoRA =-
download_civitai_model(model_id='287148', save_dir='./models/loras', filename='15_Fantasy_Classes_SD.safetensors', token=token)
# Fantasy Races LoRA: Generates characters of various fantasy races. Use race-specific keywords in your prompt.
# Keywords: Human, Elf, Orc, Air genasi, Earth genasi, Water genasi, Fire genasi, Gith, Changeling, Shifter, Beastfolk, Centaur, Merfolk, Aasimar, Android, Warforged
!echo -= Fantasy Races LoRA =-
download_civitai_model(model_id='289865', save_dir='./models/loras', filename='15_Fantasy_Races.safetensors', token=token)

## Models
# 1.5 DreamShaper - A versatile SD 1.5-based model published in July 2023 with over 13,000 positive reviews. It's known for better handling of character LoRAs, improved photorealism, and better NSFW capabilities.
!echo -= 1.5 Dreamshaper =-
!wget -c https://civitai.com/models/4384/dreamshaper -O ./models/checkpoints/15_dreamshaper_8.safetensors
# 1.5 Stable Diffusion v1.5 FP16 model: Archived version of the original SD 1.5, converted to FP16 precision with added metadata header, suitable for legacy testing and resource-constrained environments
!echo -= 1.5 Stable Diffusion v1.5 FP16 model =-
!wget -c https://huggingface.co/Comfy-Org/stable-diffusion-v1-5-archive/resolve/main/v1-5-pruned-emaonly-fp16.safetensors -o ./models/checkpoints/15_v1-5-pruned-emaonly-fp16.safetensors
# Realistic Vision V5.1 model: An improved version of the Realistic Vision series, addressing issues from V5.0 such as artifacts, facial features, and compatibility with LoRA and textual inversion. Optimized for photorealistic image generation with enhanced skin detail and overall quality.
!echo -= Realistic Vision V5.1 model =-
!wget -c https://huggingface.co/SG161222/Realistic_Vision_V5.1_noVAE/resolve/main/Realistic_Vision_V5.1.safetensors -O ./models/checkpoints/15_realistic_vision_V5.1.safetensors
# Realistic Vision V5.1 FP16 (no EMA): Memory-optimized version of RV5.1 converted to FP16 precision without EMA weights, ideal for 8GB VRAM GPUs (RTX 20/30 series). Maintains photorealism while enabling 768x768 generation on mid-range hardware.
!echo -= Realistic Vision V5.1 FP16 noEMA =-
!wget -c https://huggingface.co/SG161222/Realistic_Vision_V5.1_noVAE/resolve/main/Realistic_Vision_V5.1_fp16-no-ema.safetensors -O ./models/checkpoints/15_realistic_vision_V5.1_fp16-no-ema.safetensors
# Realistic Vision V6.0B1 w/VAE: Combined model and VAE package for enhanced photorealism at 640x960 resolutions. Optimized for single-file workflows with reduced VRAM usage.
!echo -= Realistic Vision V6.0B1 w/VAE =-
!wget -c https://civitai.com/api/download/models/128713 -O ./models/checkpoints/15_realistic_visionV60B1_v51VAE.safetensors

## unCLIP models
# SD 1.5 illuminatiDiffusionV1 unCLIP: Fine-tuned model with unCLIP capabilities for reference-based image generation, optimized with half-precision (FP16) for reduced memory usage
!echo -= SD 1.5 illuminatiDiffusionV1 unCLIP =-
!wget -c https://huggingface.co/comfyanonymous/illuminatiDiffusionV1_v11_unCLIP/resolve/main/illuminatiDiffusionV1_v11-unclip-h-fp16.safetensors -O ./models/checkpoints/15_illuminatiDiffusionV1_v11-unclip-h-fp16.safetensors
# SD 1.5 WD 1.5 Beta2 Aesthetic unCLIP: Fine-tuned model with unCLIP capabilities for reference-based image generation, optimized with half-precision (FP16) for reduced memory usage.
!echo -= SD 1.5 WD 1.5 Beta2 Aesthetic unCLIP =-
!wget -c https://huggingface.co/comfyanonymous/wd-1.5-beta2_unCLIP/resolve/main/wd-1-5-beta2-aesthetic-unclip-h-fp16.safetensors -O ./models/checkpoints/15_wd-1-5-beta2-aesthetic-unclip-h-fp16.safetensors

## upscale model
# 1.5 RealESRGAN x4plus: Powerful image upscaling model that enhances details while removing artifacts, optimized for 4x upscaling with improved performance on faces and real-world images
!echo -= 1.5 RealESRGAN x4plus =-
!wget -c https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -O ./models/upscale_models/15_RealESRGAN_x4plus.pth
# 1.5 RealESRGAN x2: Image upscaling model optimized for 2x upscaling, providing a balance between detail enhancement and artifact removal for real-world images
!echo -= 1.5 RealESRGAN x2 =-
!wget -c https://huggingface.co/sberbank-ai/Real-ESRGAN/resolve/main/RealESRGAN_x2.pth -O ./models/upscale_models/15_RealESRGAN_x2.pth
# 1.5 RealESRGAN x4: Image upscaling model designed for 4x upscaling, offering detail enhancement and artifact removal capabilities similar to x4plus but potentially with different optimization parameters
!echo -= 1.5 RealESRGAN x4 =-
!wget -c https://huggingface.co/sberbank-ai/Real-ESRGAN/resolve/main/RealESRGAN_x4.pth -O ./models/upscale_models/15_RealESRGAN_x4.pth

## VAE
# 1.5 SD VAE-FT-MSE: Improved autoencoder fine-tuned on LAION datasets with emphasis on MSE reconstruction, producing smoother outputs and better face reconstruction than the original SD 1.5 VAE
!echo -= 1.5 SD VAE-FT-MSE =-
!wget -c https://huggingface.co/stabilityai/sd-vae-ft-mse/resolve/main/diffusion_pytorch_model.safetensors -O ./models/vae/15_sd-vae-ft-mse.safetensors
# 1.5 VAE-FT-MSE-840000: Improved autoencoder fine-tuned on LAION datasets for 840,000 steps, emphasizing MSE reconstruction for smoother outputs and better face reconstruction
!echo -= 1.5 VAE-FT-MSE-840000 =-
!wget -c https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors -O ./models/vae/15_vae-ft-mse-840000-ema-pruned.safetensors
# 1.5 OrangeMix VAE: Specialized VAE for anime-style image generation, particularly designed for OrangeMix model family (AbyssOrangeMix, VividOrangeMix) to produce vibrant colors and proper skin tones
#!echo -= 1.5 OrangeMix VAE =-
#!wget -c https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/VAEs/orangemix.vae.pt -P ./models/vae/15_orangemix.vae.pt
# Realistic Vision V5.1 VAE: Specialized Variational Autoencoder designed for photorealistic outputs, reduces color artifacts and improves facial details when paired with Realistic Vision models. Optimized for SD 1.5 architecture compatibility.
!echo -= Realistic Vision V5.1 VAE =-
!wget -c https://huggingface.co/SG161222/Realistic_Vision_V5.1_noVAE/resolve/main/vae/diffusion_pytorch_model.safetensors -O ./models/VAE/15_Realistic_Vision_V5.1_VAE.safetensors

### 2.x
## Loras
# 2.1 768 Theovercomer8's Contrast Fix: LoRA for producing high-contrast, low-key images, trained on SD2.1-768 using offset noise technique.
!echo -= 2.1 768 Theovercomer8s Contrast Fix =-
!wget -c https://civitai.com/api/download/models/10350 -O ./models/loras/21_theovercomer8sContrastFix_sd21768.safetensors


### SDXL
## Loras
# SDXL 1.0 Offset Example LoRA: Official LoRA for controlling brightness, contrast, and color in SDXL 1.0 generations using noise offset technique.
!echo -= SDXL 1.0 Offset Example LoRA =-
!wget -c https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_offset_example-lora_1.0.safetensors -O ./models/loras/sdxl_sd_xl_offset_example-lora_1.0.safetensors

## Models
# SDXL DreamShaper XL - The SDXL version published in February 2024 with nearly 5,000 positive reviews. The current version is based on SDXL Turbo and should be used with CFG scale 2 and 4-8 sampling steps, preferably with DPM++ SDE Karras sampler.
!echo -= SDXL Dreamshaper XL =-
!wget -c https://civitai.com/models/112902/dreamshaper-xl -O ./models/checkpoints/sdxl_dreamshaperXL_v21TurboDPMSDE.safetensors

# SDXL ProteusV0.3: Advanced text-to-image model fine-tuned with 200,000+ anime images and DPO optimization, excelling at photorealistic and stylistic generation with enhanced facial details and lighting effects
!echo -= SDXL ProteusV0.3 =-
!wget -c https://huggingface.co/dataautogpt3/ProteusV0.3/resolve/main/ProteusV0.3.safetensors -O ./models/checkpoints/sdxl_ProteusV0.3.safetensors

# SDXL Base 1.0: Stable Diffusion XL base model that generates initial latents with 3x larger UNet, dual text encoders, and improved capabilities for realistic faces, legible text, and better composition.
!echo -= SDXL Base 1.0 =-
!wget -c https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0.safetensors -O ./models/checkpoints/sdxl_sd_xl_base_1.0.safetensors

# SDXL Refiner 1.0: Specialized second-stage model that enhances outputs from the base model, implementing an ensemble of experts approach to improve visual fidelity and fix low-quality details like deformed faces.
!echo -= SDXL Refiner 1.0 =-
!wget -c https://huggingface.co/stabilityai/stable-diffusion-xl-refiner-1.0/resolve/main/sd_xl_refiner_1.0.safetensors -O ./models/checkpoints/sdxl_sd_xl_refiner_1.0.safetensors

## Clip Vision
# SDXL CLIP Vision G: Large-scale vision encoder model (3.44GB) used in unCLIP workflows for visual encoding, enabling image-based conditioning and reference-guided generation in ComfyUI.
!echo -= SDXL CLIP Vision G =-
!wget -c https://huggingface.co/comfyanonymous/clip_vision_g/resolve/main/clip_vision_g.safetensors -O ./models/clip_vision/sdxl_clip_vision_g.safetensors

## Control-lora
# SDXL Control-LoRA Canny: Efficient edge-detection control model (738MB) that follows image edges for guided generation, significantly smaller than traditional 4.7GB ControlNets
!echo -= SDXL Control-LoRA Canny =-
!wget -c https://huggingface.co/stabilityai/control-lora/resolve/main/control-LoRAs-rank256/control-lora-canny-rank256.safetensors -O ./models/control-lora/sdxl_control-lora-canny-rank256.safetensors
# SDXL Control-LoRA Depth: Depth-map control model using MiDaS and ClipDrop depth estimation to guide generation based on spatial relationships, optimized to 738MB for consumer GPUs
!echo -= SDXL Control-LoRA Depth =-
!wget -c https://huggingface.co/stabilityai/control-lora/resolve/main/control-LoRAs-rank256/control-lora-depth-rank256.safetensors -O ./models/control-lora/sdxl_control-lora-depth-rank256.safetensors
# SDXL Control-LoRA Recolor: Specialized model for colorizing black and white photographs while maintaining high-quality control capabilities, compressed to 738MB through LoRA technology
!echo -= Control-LoRA Recolor =-
!wget -c https://huggingface.co/stabilityai/control-lora/resolve/main/control-LoRAs-rank256/control-lora-recolor-rank256.safetensors -O ./models/control-lora/sdxl_control-lora-recolor-rank256.safetensors
# SDXL Control-LoRA Sketch: White-on-black drawing colorizer designed specifically for sketches and line art, reduced to 738MB from the original 4.7GB ControlNet size
!echo -= Control-LoRA Sketch =-
!wget -c https://huggingface.co/stabilityai/control-lora/resolve/main/control-LoRAs-rank256/control-lora-sketch-rank256.safetensors -O ./models/control-lora/sdxl_control-lora-sketch-rank256.safetensors


### I recommend these workflow examples: https://comfyanonymous.github.io/ComfyUI_examples/sdxl/

### IP-Adapter
## IP-Adapter Models
# IP-Adapter Models
!wget -c https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-plus_sd15.bin -P ./models/ipadapter/
# Basic model, average strength
!wget -c https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter_sd15.safetensors -P ./models/ipadapter/
# Light impact model
!wget -c https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter_sd15_light_v11.bin -P ./models/ipadapter/
# Plus model, very strong
!wget -c https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-plus_sd15.safetensors -P ./models/ipadapter/
# Face model, portraits
!wget -c https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-plus-face_sd15.safetensors -P ./models/ipadapter/
# Stronger face model, not necessarily better
!wget -c https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-full-face_sd15.safetensors -P ./models/ipadapter/
# Base model, requires bigG clip vision encoder
!wget -c https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter_sd15_vit-G.safetensors -P ./models/ipadapter/
# SDXL model
!wget -c https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/ip-adapter_sdxl_vit-h.safetensors -P ./models/ipadapter/
# SDXL plus model
!wget -c https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/ip-adapter-plus_sdxl_vit-h.safetensors -P ./models/ipadapter/
# SDXL face model
!wget -c https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/ip-adapter-plus-face_sdxl_vit-h.safetensors -P ./models/ipadapter/
# vit-G SDXL model, requires bigG clip vision encoder
!wget -c https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/ip-adapter_sdxl.safetensors -P ./models/ipadapter/

# IP-Adapter FaceID Models - FaceID models require insightface, you need to install it in your ComfyUI environment. Check this issue for help. Remember that most FaceID models also need a LoRA.
# base FaceID model
!wget -c https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid_sd15.bin -P ./models/ipadapter/
# FaceID plus v2
!wget -c https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-plusv2_sd15.bin -P ./models/ipadapter/
# text prompt style transfer for portraits
!wget -c https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-portrait-v11_sd15.bin -P ./models/ipadapter/
# SDXL base FaceID
!wget -c https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid_sdxl.bin -P ./models/ipadapter/
# SDXL plus v2
!wget -c https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-plusv2_sdxl.bin -P ./models/ipadapter/
# SDXL text prompt style transfer
!wget -c https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-portrait_sdxl.bin -P ./models/ipadapter/
# very strong style transfer SDXL only
!wget -c ip-adapter-faceid-portrait_sdxl_unnorm.bin -P ./models/ipadapter/
# IP-Adapter - community - general composition ignoring style and content, more about it at https://huggingface.co/ostris/ip-composition-adapter
!wget -c https://huggingface.co/ostris/ip-composition-adapter/resolve/main/ip_plus_composition_sd15.safetensors -P ./models/ipadapter/
# SDXL version
!wget -c https://huggingface.co/ostris/ip-composition-adapter/resolve/main/ip_plus_composition_sdxl.safetensors -P ./models/ipadapter/

## IP-Adapter Loras
# IP-Adapter - FaceID - Most FaceID models require a LoRA. If you use the IPAdapter Unified Loader FaceID it will be loaded automatically if you follow the naming convention. Otherwise you have to load them manually, be careful each FaceID model has to be paired with its own specific LoRA.
!wget -c https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-plus_sd15_lora.safetensors -P ./models/loras/
!wget -c https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-plusv2_sd15_lora.safetensors -P ./models/loras/
# SDXL FaceID LoRA
!wget -c https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid_sdxl_lora.safetensors -P ./models/loras/
# SDXL plus v2 LoRA
!wget -c https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-plusv2_sdxl_lora.safetensors -P ./models/loras/

## IP-Adapter CLIPVision models
# 15 CLIP-ViT-H-14-laion2B-s32B-b79K: Large-scale vision-language model with ViT-H/14 architecture, trained on 2 billion image-text pairs for zero-shot classification and image-text retrieval tasks
!echo -= 15 CLIP-ViT-H-14-laion2B-s32B-b79K =-
!wget -c https://huggingface.co/h94/IP-Adapter/resolve/main/models/image_encoder/model.safetensors -O ./models/clip_vision/15_CLIP-ViT-H-14-laion2B-s32B-b79K.safetensors
# SDXL CLIP-ViT-bigG-14-laion2B-39B-b160K: Advanced vision-language model with ViT-bigG/14 backbone, trained on 39 billion image-text pairs, achieving 80.1% zero-shot accuracy on ImageNet-1k
!echo -= SDXL CLIP-ViT-bigG-14-laion2B-39B-b160K =-
!wget -c https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/image_encoder/model.safetensors -O ./models/clip_vision/sdxl_CLIP-ViT-bigG-14-laion2B-39B-b160k.safetensors


# SD2
#!wget -c https://huggingface.co/stabilityai/stable-diffusion-2-1-base/resolve/main/v2-1_512-ema-pruned.safetensors -P ./models/checkpoints/
#!wget -c https://huggingface.co/stabilityai/stable-diffusion-2-1/resolve/main/v2-1_768-ema-pruned.safetensors -P ./models/checkpoints/

# Some SD1.5 anime style
#!wget -c https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix2/AbyssOrangeMix2_hard.safetensors -P ./models/checkpoints/
#!wget -c https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3A1_orangemixs.safetensors -P ./models/checkpoints/
#!wget -c https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3A3_orangemixs.safetensors -P ./models/checkpoints/
#!wget -c https://huggingface.co/Linaqruf/anything-v3.0/resolve/main/anything-v3-fp16-pruned.safetensors -P ./models/checkpoints/

# Waifu Diffusion 1.5 (anime style SD2.x 768-v)
#!wget -c https://huggingface.co/waifu-diffusion/wd-1-5-beta3/resolve/main/wd-illusion-fp16.safetensors -P ./models/checkpoints/15_wd-1-5-beta3/resolve/main/wd-illusion-fp16.safetensors






# Controlnet Preprocessor nodes by Fannovel16
!cd custom_nodes && git clone https://github.com/Fannovel16/comfy_controlnet_preprocessors; cd comfy_controlnet_preprocessors && python install.py


# Download and extract InsightFace buffalo_l models
!echo -= Downloading InsightFace buffalo_l models =-
!wget -c https://github.com/deepinsight/insightface/releases/download/v0.7/buffalo_l.zip -O /content/drive/MyDrive/ComfyUI/models/insightface/buffalo_l.zip
!unzip -o /content/drive/MyDrive/ComfyUI/models/insightface/buffalo_l.zip -d /content/drive/MyDrive/ComfyUI/models/insightface/
!rm /content/drive/MyDrive/ComfyUI/models/insightface/buffalo_l.zip


# Download required SDXL LCM LoRA model (rename as required by Comfy_Dungeon)
# SDXL LCM-LoRA: Latent Consistency Model adapter that dramatically reduces inference steps to 2-8 while maintaining quality, requiring guidance scale values between 1.0-2.0 for optimal results
!echo -= SDXL LCM-LoRA for Comfy_Dungeon =-
!wget -c https://huggingface.co/latent-consistency/lcm-lora-sdxl/resolve/main/pytorch_lora_weights.safetensors -O /content/drive/MyDrive/ComfyUI/models/loras/lcm-lora-sdxl.safetensors

!echo -= Done =-


Mounting Google Drive...
/
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/ComfyUI
-= 1.4 GLIGEN Textbox =-
--2025-03-29 16:22:43--  https://huggingface.co/comfyanonymous/GLIGEN_pruned_safetensors/resolve/main/gligen_sd14_textbox_pruned_fp16.safetensors
Resolving huggingface.co (huggingface.co)... 18.239.50.103, 18.239.50.80, 18.239.50.49, ...
Connecting to huggingface.co (huggingface.co)|18.239.50.103|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.hf.co/repos/72/d5/72d59405656f42915470665ee8ae163df85bcc1b645aa89993aa6f585f59065f/3183c006351c033c3b183ac0a2c35f72a79c9ec0fcba3df9a05afd4320d116b2?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27gligen_sd14_textbox_pruned_fp16.safetensors%3B+filename%3D%22gligen_sd14_textbox_pruned_fp16.safetensors%22%3B&Expires=1743268963&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVz

### Run ComfyUI with cloudflared (Recommended Way)




In [ ]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

import subprocess
import threading
import time
import socket
import urllib.request

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch cloudflared (if it gets stuck here cloudflared is having issues)\n")

  p = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  for line in p.stderr:
    l = line.decode()
    if "trycloudflare.com " in l:
      print("This is the URL to access ComfyUI:", l[l.find("http"):], end='')
    #print(l, end='')


threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server

--2025-03-29 16:32:05--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2025.2.1/cloudflared-linux-amd64.deb [following]
--2025-03-29 16:32:05--  https://github.com/cloudflare/cloudflared/releases/download/2025.2.1/cloudflared-linux-amd64.deb
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://objects.githubusercontent.com/github-production-release-asset-2e65be/106867604/4f284b8f-4452-49e8-a0c0-0e45c3b6e65e?X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Credential=releaseassetproduction%2F20250329%2Fus-east-1%2Fs3%2Faws4_request&X-Amz-Date=20250329T163205Z&X-Amz-Expires=300&X-Amz-Signature=6f0868000ec9f17bb9b42c8b0dce39874f4500878000d5200f04904b1cf0